In [1]:
from utils import LoginCredentials
import wandb

authenticator = LoginCredentials()

wandb.login(key=authenticator.wandb_key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: finnhenri-smidt. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/henrismidt/.netrc


True

In [2]:
import torch
from torchvision import transforms
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from transformers import MobileViTImageProcessor
import wandb

from dataset import MRIImageDataModule, MRIDataset
from models import MobileViTLightning, EfficientNetBaseline
from utils import get_best_device, LoginCredentials, set_reproducibility
from sampler import WeightedRandomSampler

from datetime import datetime
import lightning.pytorch as pl
from sklearn.metrics import f1_score
from sklearn.metrics import (
    precision_score,
    recall_score,
    classification_report,
    confusion_matrix,
)
import os
import pickle

set_reproducibility(42)

wandb.finish()  # Make sure previous sessions are finished


csv_path = "Data/metadata_for_preprocessed_files.csv"

# Define sweep configuration
sweep_config = {
    "method": "grid",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "model_name": {
            "values": ["mobilevit-s", "efficientnet-b2"]
        },  # ["efficientnet-b2""mobilevit-s", "efficientnet-b0", "efficientnet-b2", "efficientnet-b5"]
        "slice_number": {
            "values": [
                "65",
                "86",
                "56",
                "95",
                "62",
                "35",
                "59",
                "74",
                "80",
                "134",
            ]  # ['65', '86', '56', '95', '62', '35', '59', '74', '80', '134', '41', '104', '101', '116', '68', '89', '107', '92', '71', '77', '113', '23', '98', '110', '131', '128', '125', '122', '119', '20', '83', '53', '50', '47', '44', '38', '32', '29', '26', '137']
        },
        "learning_rate": {"values": [0.0001]},
        "batch_size": {"values": [40]},
        "epochs": {"values": [60]},
        "sampling_strategy": {"values": ["custom"]},  #'inverse', 'sqrt',
        "sampling_weights": {
            "values": [[0.34, 0.74, 0.4, 0.44]]
        },  # Single parameter for all class weights
        # Average of top 3 weights for model MobileVit based on val_loss:
        # [0.47, 0.8, 0.79, 0.43]
        # Average of top 3 weights for model efficientnet-b2 based on val_loss:
        # [0.34, 0.74, 0.4, 0.44]
        "self_distillation_alpha": {"values": [0.3]},
        "self_distillation_temperature": {"values": [3]},
        "smoothing": {"values": [None]},
    },
}

sweep_id = wandb.sweep(sweep=sweep_config, project="Alzheimer-Detection")


def load_soft_labels(slice_number, model_name):
    soft_labels_path = (
        f"soft_labels/custom_sampler/{model_name}/soft_labels_slice_{slice_number}.pkl"
    )
    if os.path.exists(soft_labels_path):
        with open(soft_labels_path, "rb") as f:
            soft_labels = pickle.load(f)
    else:
        soft_labels = None
    return soft_labels


# Define the training function


def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        device = get_best_device()

        if config.model_name == "mobilevit-s":
            # Load the preprocessor
            model_ckpt = "apple/mobilevit-small"
            processor = MobileViTImageProcessor.from_pretrained(model_ckpt)

            def transform(image):
                # Use MobileViTImageProcessor for preprocessing
                return processor(image, return_tensors="pt")["pixel_values"].squeeze(0)

            model = MobileViTLightning(
                model_ckpt=model_ckpt,
                num_labels=4,
                self_distillation_alpha=config.self_distillation_alpha,
                self_distillation_temperature=config.self_distillation_temperature,
            )

        elif config.model_name.startswith("efficientnet"):
            transform = transforms.Compose(
                [
                    transforms.ToPILImage(),
                    transforms.Resize(224),
                    transforms.ToTensor(),
                    transforms.Normalize(
                        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                    ),
                ]
            )
            # transform = None
            model = EfficientNetBaseline(
                model_name=config.model_name,
                lr=config.learning_rate,
                self_distillation_alpha=config.self_distillation_alpha,
                self_distillation_temperature=config.self_distillation_temperature,
            )

        # Load soft labels
        soft_labels = load_soft_labels(int(config.slice_number), config.model_name)

        data_module = MRIImageDataModule(
            csv_path,
            slice_number=int(config.slice_number),
            transform=transform,
            batch_size=config.batch_size,
            num_workers=0,
            soft_labels=soft_labels,
        )
        data_module.setup()
        custom_weights = dict(zip(range(4), config.sampling_weights))
        train_loader = data_module.train_dataloader(
            sampling_strategy=config.sampling_strategy,
            smoothing=config.smoothing,
            custom_weights=custom_weights,
        )
        val_loader = data_module.val_dataloader()
        test_loader = data_module.test_dataloader()

        wandb_logger = WandbLogger()

        checkpoint_callback = ModelCheckpoint(
            dirpath=f"model_checkpoints/student_models/{config.model_name}",
            filename=f"slice_number_{config.slice_number}_sd_alpha_{config.self_distillation_alpha}_sd_temp_{config.self_distillation_temperature}",
            monitor="val_loss",
            mode="min",
            save_top_k=1,
        )

        trainer = L.Trainer(
            max_epochs=config.epochs,
            devices="auto",
            accelerator="auto",
            logger=wandb_logger,
            callbacks=[checkpoint_callback],
            log_every_n_steps=24,
        )

        trainer.fit(
            model=model, train_dataloaders=train_loader, val_dataloaders=val_loader
        )

        # Load best model for testing
        best_model_path = checkpoint_callback.best_model_path
        if config.model_name == "mobilevit-s":
            model_ckpt = "apple/mobilevit-small"
            best_model = MobileViTLightning.load_from_checkpoint(
                best_model_path, model_ckpt=model_ckpt, num_labels=4
            )

        elif config.model_name.startswith("efficientnet"):
            best_model = EfficientNetBaseline.load_from_checkpoint(
                best_model_path,
                model_name=config.model_name,
                num_classes=4,
            )

        # Evaluate on test set
        best_model = best_model.to(device)
        best_model.eval()
        all_preds = []
        all_labels = []
        all_ids = []
        with torch.no_grad():
            for batch in test_loader:
                inputs = batch["inputs"]
                labels = batch["labels"]
                id = batch["id"]
                inputs = inputs.to(device).float()
                outputs = best_model(inputs)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_ids.extend(id)

            # Calculate metrics
            f1_weighted = f1_score(all_labels, all_preds, average="weighted")
            f1_individual = f1_score(all_labels, all_preds, average=None)
            precision = precision_score(all_labels, all_preds, average="weighted")
            recall = recall_score(all_labels, all_preds, average="weighted")
            conf_matrix = confusion_matrix(all_labels, all_preds)
            classification_rep = classification_report(
                all_labels, all_preds, output_dict=True
            )

            # Log metrics to wandb
            wandb.log(
                {
                    "test_f1_weighted": f1_weighted,
                    "test_f1_individual": f1_individual,
                    "test_precision": precision,
                    "test_recall": recall,
                    "confusion_matrix": conf_matrix,
                    "classification_report": classification_rep,
                }
            )


# Run the sweep
# wandb.agent(sweep_id, function=train)

Seed set to 42


Create sweep with ID: onh2y60k
Sweep URL: https://wandb.ai/finnhenri-smidt/Alzheimer-Detection/sweeps/onh2y60k


In [3]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: bpj3ip1g with config:
wandb: 	batch_size: 40
wandb: 	epochs: 60
wandb: 	learning_rate: 0.0001
wandb: 	model_name: mobilevit-s
wandb: 	sampling_strategy: custom
wandb: 	sampling_weights: [0.34, 0.74, 0.4, 0.44]
wandb: 	self_distillation_alpha: 0.3
wandb: 	self_distillation_temperature: 3
wandb: 	slice_number: 65
wandb: 	smoothing: None
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MobileViTForImageClassification were not initialized from the model checkpoint at apple/mobilevit-small and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 640]) in the checkpoint and torch.Size([4, 640]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Us

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=24). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=60` reached.


Loaded pretrained weights for efficientnet-b2


/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_star

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_f1_weighted,▁
test_precision,▁
test_recall,▁
train_loss,█▄▄▄▂▂▂▂▅▂▂▂▂▂▂▂▁▂▂▂▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▆▄▃▄▃▂▂▂▂▂▂▂▁▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,59
test_f1_weighted,0.80944
test_precision,0.78606
test_recall,0.83636


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
